In [35]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import statsmodels.api as sm
import matplotlib.pyplot as plt
import xgboost as xgb
import warnings


### Data loader

In [2]:
df = pd.read_csv(
    '../data/processed/data-train/input/S0000137-Ngoc Khanh.csv')


In [15]:
data = df['PM2.5'].values

In [14]:
from statsmodels.tsa.stattools import adfuller
adf, pval, usedlag, nobs, crit_vals, icbest = adfuller(df['PM2.5'].values)
print('ADF test statistic:', adf)
print('ADF p-values:', pval)
print('ADF number of lags used:', usedlag)
print('ADF number of observations:', nobs)
print('ADF critical values:', crit_vals)
print('ADF best information criterion:', icbest)


ADF test statistic: -6.435524204964674
ADF p-values: 1.6565574828994273e-08
ADF number of lags used: 27
ADF number of observations: 8972
ADF critical values: {'1%': -3.4310790650823195, '5%': -2.8618621993323705, '10%': -2.566941501681359}
ADF best information criterion: 67397.21075403543


In [23]:
X_data = []
y_data = []
for i in range(len(data) - 169):
    X_data.append(data[i:i+168])
    y_data.append(data[i+168])

X_windows = np.vstack(X_data)

### Cost function

In [3]:
# def mdape(preds, target):
#     mdape = np.median((np.abs(np.subtract(target, preds)) / preds))
#     return mdape


# def lgbm_mdape(preds, train_data):
#     labels = train_data.get_label()
#     mdape_val = mdape(np.array(preds), np.array(labels))
#     return 'MDAPE', mdape, False


### Train test split

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_windows, np.array(y_data), test_size=0.2, random_state=1)


In [4]:
# def split(df, ratio):
#     n = len(df)
#     train = df.iloc[: int(ratio * n), ]
#     test = df.iloc[int(ratio * n) :, ]

#     return train, test


In [5]:
# train, test = split(df, 0.8)
# train, val = split(train, 0.8)

In [6]:
# cols = [col for col in df.columns if col not in ['timestamp', 'PM2.5', 'year', 'humidity', 'temperature']]

In [7]:
# Y_train = train['PM2.5']
# X_train = train[cols]

# Y_val = val['PM2.5']
# X_val = val[cols]


### Base model

In [8]:
# lgb_params = {'metric': {'mdape'},
#               'num_leaves': 10,
#               'learning_rate': 0.02,
#               'feature_fraction': 0.8,
#               'max_depth': 5,
#               'verbose': 0,
#               'num_boost_round': 15000,
#               'early_stopping_rounds': 200,
#               'nthread': -1}


In [9]:
# lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
# lgbval = lgb.Dataset(data=X_val, label=Y_val,
#                      reference=lgbtrain, feature_name=cols)


In [37]:
def mdape(preds, target):
    mdape = np.median((np.abs(np.subtract(target, preds)) / preds)) * 100
    return mdape


def lgbm_mdape(preds, train_data):
    labels = train_data.get_label()
    mdape_val = mdape(np.array(preds), np.array(labels))
    return 'MDAPE', mdape_val, False


In [ ]:
# model = lgb.train(lgb_params, lgbtrain,
#                   valid_sets=[lgbtrain, lgbval],
#                   num_boost_round=lgb_params['num_boost_round'],
#                   early_stopping_rounds=lgb_params['early_stopping_rounds'],
#                   feval=lgbm_mdape,
#                   verbose_eval=200)


In [12]:
# y_pred_val = model.predict(X_val)

In [13]:
# mdape(y_pred_val, Y_val)

37.16799319193174

In [ ]:
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import KFold
# # import neptune.new as neptune
# from sklearn.metrics import mean_squared_error

# # specify the grid for the grid search of hyperparameter tuning
# # parameters = {'max_depth': list(range(2, 20, 4)),
# #               'gamma': list(range(0, 10, 2)),
# #               'min_child_weight': list(range(0, 10, 2)),
# #               'eta': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5]
# #               }

# # param_list = [(x, y, z, a) for x in parameters['max_depth'] for y in parameters['gamma']
# #               for z in parameters['min_child_weight'] for a in parameters['eta']]


# for params in param_list:

#     mses = []

#     # run = neptune.init(
#     #     project="YOU/YOUR_PROJECT",
#     #     api_token="YOUR_API_TOKEN",
#     # )

#     # run['params'] = params

#     my_kfold = KFold(n_splits=10, shuffle=True, random_state=0)

#     for train_index, test_index in my_kfold.split(X_windows):

#         X_train, X_test = X_windows[train_index], X_windows[test_index]
#         y_train, y_test = np.array(
#             y_data)[train_index], np.array(y_data)[test_index]

#         xgb_model = xgb.XGBRegressor(
#             max_depth=params[0], gamma=params[1], min_child_weight=params[2], eta=params[3])
#         xgb_model.fit(X_train, y_train)
#         preds = xgb_model.predict(X_test)

#         mses.append(mean_squared_error(y_test, preds))

#     average_mse = np.mean(mses)
#     std_mse = np.std(mses)
#     # run['average_mse'] = average_mse
#     # run['std_mse'] = std_mse

#     # run.stop()


In [29]:
# X_test.shape

(168,)

In [40]:

model = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.01)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          early_stopping_rounds=20)



[0]	validation_0-rmse:70.09584	validation_1-rmse:71.53015
[1]	validation_0-rmse:69.41863	validation_1-rmse:70.84229
[2]	validation_0-rmse:68.74839	validation_1-rmse:70.16155
[3]	validation_0-rmse:68.08502	validation_1-rmse:69.48784
[4]	validation_0-rmse:67.42847	validation_1-rmse:68.81836
[5]	validation_0-rmse:66.77863	validation_1-rmse:68.15590
[6]	validation_0-rmse:66.13546	validation_1-rmse:67.50196
[7]	validation_0-rmse:65.49892	validation_1-rmse:66.85293
[8]	validation_0-rmse:64.86874	validation_1-rmse:66.21209
[9]	validation_0-rmse:64.24509	validation_1-rmse:65.57807
[10]	validation_0-rmse:63.62789	validation_1-rmse:64.94886
[11]	validation_0-rmse:63.01677	validation_1-rmse:64.32872
[12]	validation_0-rmse:62.41215	validation_1-rmse:63.71253
[13]	validation_0-rmse:61.81379	validation_1-rmse:63.10253
[14]	validation_0-rmse:61.22110	validation_1-rmse:62.50168
[15]	validation_0-rmse:60.63496	validation_1-rmse:61.90436
[16]	validation_0-rmse:60.05422	validation_1-rmse:61.31722
[17]	va

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [33]:
preds

array([52.653793], dtype=float32)